In [1]:
import pandas as pd

raw = pd.read_csv('cordlife_faqs2.csv')

lookup = {}
for a, b, *_ in raw.values:
    lookup[a] = b

raw

,prompt,response,Unnamed: 2
0,What is cord blood?,Cord blood is blood that remains in the umbili...,NaN
1,What are cord blood stem cells?,Cord blood stem cells are also known as Haemat...,NaN
2,Why should I save my babys cord blood stem cells?,There are several advantages of storing your b...,NaN
3,What can cord blood stem cells do?,Cord blood stem cells can:\n\nReplace and rege...,NaN
4,How does a cord blood stem cell transplant work?,The purpose of a stem cell treatment is to rec...,NaN
...,...,...,...
64,??????????,�????????????????Haematopoietic Stem Cells?HSC...,NaN
65,????????????????,�????????????????????\n\n????????????????????\...,NaN
66,Apakah darah tali pusat?,Darah tali pusat ialah darah yang kekal dalam ...,NaN
67,Apakah sel punca darah tali pusat?,"""Sel punca darah tali pusat juga dikenali seba...",NaN


In [2]:
# sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
from sentence_transformers import SentenceTransformer, util

sentences = list(raw['prompt'].values)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
sentences_embeddings = [model.encode(s, convert_to_tensor=True) for s in sentences]


c:\Users\amir.rahman\AppData\Local\anaconda3\envs\cordlife\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def search(q, THRESHOLD=0.65):
    q_embedding = model.encode(q, convert_to_tensor=True)
    out = []
    for sentence, sentence_embedding in zip(sentences, sentences_embeddings):
        score = util.pytorch_cos_sim(q_embedding, sentence_embedding).item()
        out.append((sentence, score))
    out.sort(key=lambda x:-x[-1])
    if out[0][1] < THRESHOLD:
        return ['I don\'t know']
    top = out[:3]
    return [(qns, lookup[qns], score) for qns,score in top]

search('what is cordlife preferential plan and how would it benefit a parent like me')

[('How would Preferential Plan benefit a Cordlife Parent like me?',
  "As a gesture of our appreciation to our Cordlife parents, we've introduced the Preferential Plan which allows Cordlife Preferential Plan lets you enjoy: Protection against increases in Annual Instalment Fees. Protection against hikes in Goods and Services Tax (GST). The convenience of a one-time payment for the remaining balance of annual instalment fee. Multiple payment modes such as Child Development Account (CDA), credit card interest-free instalment plans & etc.                                                           Contact us to find out more about the full benefits.",
  0.9360084533691406),
 ('What is Cordlife Preferential Plan?',
  "As a gesture of our appreciation to our Cordlife parents, we've introduced the Preferential Plan which allows parents to prepay the balance of their annual fees. Multiple payment modes are available; including the use of your child's Child Development Account (CDA) for this one

In [31]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import langid

# Load data from XLSX file
file_path = 'cordlife_faqs1 - Copy.xlsx'
raw = pd.read_excel(file_path)

# Extract prompts and responses
prompts = [str(prompt) for prompt in raw['prompt'].values]  # Convert to string
responses = list(raw['response'].values)

# Load Sentence Transformer model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Encode prompts into sentence embeddings
prompt_embeddings = [model.encode(prompt, convert_to_tensor=True) for prompt in prompts]

def search_multilingual(query, THRESHOLD=0.65):
    lang, _ = langid.classify(query)  # Detect the language of the query
    q_embedding = model.encode(query, convert_to_tensor=True)

    out = []
    for prompt, prompt_embedding, response in zip(prompts, prompt_embeddings, responses):
        score = util.pytorch_cos_sim(q_embedding, prompt_embedding).item()
        out.append((prompt, response, score))
    out.sort(key=lambda x: -x[-1])

    if out[0][2] < THRESHOLD:
        return ['I don\'t know']

    top = out[:3]
    return [(prompt, response, score) for prompt, response, score in top]



In [32]:
# Example usage
query = '什麼是臍帶血？'
result = search_multilingual(query)
display(result)


[('什麼是臍帶血？',
  '臍帶血是嬰兒出生後和臍帶被切斷后殘留在臍帶中的血液。在懷孕期間，臍帶是母親和孩子之間的生命線。嬰兒分娩后，臍帶中的臍帶血可以為孩子和家庭成員帶來希望。臍帶血含有豐富的救命幹細胞來源，稱為造血幹細胞（HSC）。',
  1.0),
 ('What is cord blood?',
  'Cord blood is blood that remains in the umbilical cord following the birth of a baby and after the umbilical cord is cut. During pregnancy, the umbilical cord functions as a lifeline between mother and child. After a baby’s delivery, the cord blood present in the umbilical cord could offer hope for the child as well as members of the family. Cord blood contains a rich source of lifesaving stem cells called Haematopoietic Stem Cells (HSCs).',
  0.8569766879081726),
 ('Apakah darah tali pusat?',
  'Darah tali pusat adalah darah yang kekal dalam tali pusat selepas kelahiran bayi dan selepas tali pusat dipotong. Semasa mengandung, tali pusat berfungsi sebagai talian hayat antara ibu dan anak. Selepas melahirkan bayi, darah tali pusat yang terdapat dalam tali pusat boleh memberikan harapan kepada kanak-kanak itu serta ahli keluarga. D

In [29]:
# Example usage with the multilingual lookup dictionary
query = 'apakah itu CDA dan bagaimana ia berfungsi?'
result = search_multilingual(query)

# Print the result
display(result)


[('Apakah Geran Langkah Pertama CDA?',
  'Geran Langkah Pertama CDA ialah geran baru $3,000 bagi kanak-kanak Singapura yang layak yang dilahirkan mulai 24 Mac 2016. Ia dibayar ke dalam CDA kanak-kanak dan merupakan sebahagian daripada keseluruhan had sumbangan Kerajaan yang sedia ada.',
  0.8305612802505493),
 ('什麼是 CDA 第一步補助金？',
  'CDA 第一步補助金是為 2016 年 3 月 24 日出生的符合條件的新加坡兒童提供 3,000 加元的新補助金。它支付到兒童的CDA中，並構成現有政府總供款上限的一部分。',
  0.7594665288925171),
 ('兒童發展帳戶（CDA）如何運作？', 'CDA 有兩個組成部分：CDA 第一步贈款和政府美元對美元匹配。', 0.7591106295585632)]

In [28]:
query = 'kenapa saya perlu menyimpan darah tali pusat bayi saya?'
result = search_multilingual(query)

# Print the result
display(result)


[('Bagaimanakah darah tali pusat bayi saya disimpan?',
  'Darah tali pusat bayi anda disimpan dalam kantung penyimpanan kriogenik yang diluluskan oleh FDA AS yang diperbuat daripada bahan khas yang direka khusus untuk menahan suhu kriogenik. Kantung ini mempunyai dua segmen utama (20% dan 80%) yang dilampirkan secara integral, dan dua segmen ujian yang juga dilampirkan secara integral. Segmen penting membolehkan ujian tambahan pada unit yang berkaitan untuk memastikan tiada sampel bercampur dan darah tali pusat tetap berdaya maju. Ujian ini biasanya dilakukan sebelum pemindahan. Mempunyai segmen dwi bersepadu menangani kemungkinan pengembangan sel stem masa depan. Ini bermakna apabila pengembangan sel stem boleh didapati secara komersial, anda akan dapat mengeluarkan sebahagian daripada sel stem untuk pengembangan sambil mengekalkan baki dalam simpanan.',
  0.9090284705162048),
 ('Bagaimanakah darah tali pusat bayi saya dikumpulkan oleh OBGYN saya?',
  'Sejurus selepas kelahiran bayi a